## Laboratorio 2
### Detección de SPAM
> #### Gerardo Méndez 18239, Luis Pedro Cuéllar 18220 

In [1]:
import pandas as pd
import numpy as np
import re

%matplotlib inline

In [2]:
csa_df = pd.read_csv('datasets/completeSpamAssassin.csv')
ess_df = pd.read_csv('datasets/enronSpamSubset.csv')